In [246]:
# Teemu mac kernel python 3.11.7

import datetime as dt
import pandas as pd
import numpy as np
from scipy.stats import norm

In [247]:
def calculate_delta(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    return norm.cdf(d1)

In [248]:
# Black-Scholes Call Price
def black_scholes_call(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    d2 = d1 - sigma * np.sqrt(T-t)
    return S * norm.cdf(d1) - K * np.exp(-r * (T-t)) * norm.cdf(d2)

In [249]:
def interval_func(start, end, interval):
    result = []
    current = start
    while current <= end:
        result.append(round(current, 4))
        current += interval
    return result

In [250]:
def calc_delta(S, K, T, t, r, C_0, eps=np.finfo(float).eps, tol=0.1):

    for sigma in interval_func(0, 5, 0.001):
        c = black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
        if abs(c-C_0) <= tol:
            delta = calculate_delta(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
            print(f"Delta = {delta}, sigma = {sigma}, Price: {S}, S-C={S-C_0} \ncall price from BS = {round(c, 5)}, Call price from data = {C_0}")
            return delta
    print(f"Delta = 1, sigma = 0, call from BS if sigma = 0: {black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=0, eps=eps)} Call price from data = {C_0}")
    return 1

In [251]:
# Alkuarvot
# Data:
data = pd.read_feather('apple.feather')

In [264]:
# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

start_date = dt.datetime.strptime("2024-09-10", '%Y-%m-%d')
maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')

# ATM 2024-09-10
T = 45/365
t = 0
K = 220

strike = f'C{K}'
C_0 = float(df[strike].iloc[0])
S_0 = float(df['Underlying'].iloc[0])

# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps
delta = calc_delta(S=S_0, K=K, T=T, t=t, r=r, C_0=C_0, eps=np.finfo(float).eps, tol=0.1)
print(f"Delta at S_O= {delta}, S_0 = {S_0}, C_0 = {C_0}")

# Hedging interval
interval = 4


Delta = 0.5473233637793574, sigma = 0.254, Price: 220.11, S-C=211.46 
call price from BS = 8.55149, Call price from data = 8.65
Delta at S_O= 0.5473233637793574, S_0 = 220.11, C_0 = 8.65


In [267]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

# init portfolia values
OP_0 = C_0

# delta * S_0
RE_0 = -delta * S_0
print(OP_0, RE_0, OP_0+RE_0)

def hedging(interval, strike, df, delta):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    t = 0
    n = 0
    
    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
        
        if row.name == 0:
            #print(1)
            continue

        OP = c_1 - c_0

        #print(f'{delta} * ({s_1}-{s_0})')
        RE = -delta * (s_1-s_0)
        #print(f'OP: {OP}, RE: {RE}')
        A = OP + RE
    
        # print(f'Error/Difference OP + RE: {A}')
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          #print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(n-1)
    #print(n)
    return mse


8.65 -120.47134560147437 -111.82134560147436


In [268]:
hedging(interval=interval, strike=strike, df=df, delta=delta)


Error/Difference OP + RE: -0.44567457763735274
Error/Difference OP + RE: -0.2602055700157361
Error/Difference OP + RE: -0.25222269177956824
Error/Difference OP + RE: -0.1675416118435673
Delta = 0.4540964312500928, sigma = 0.231, Price: 216.32, S-C=210.87 
call price from BS = 5.37185, Call price from data = 5.45
Error/Difference OP + RE: -0.20342532268754332
Error/Difference OP + RE: 0.3490239181246355
Error/Difference OP + RE: 1.2554911923742385
Error/Difference OP + RE: -0.18575539106243083
Delta = 0.7327404342973466, sigma = 0.228, Price: 228.2, S-C=216.14 
call price from BS = 11.96892, Call price from data = 12.06
Error/Difference OP + RE: -0.4523590486655986
Error/Difference OP + RE: -0.34946639086761555
Error/Difference OP + RE: 0.012740434297345948
Error/Difference OP + RE: -0.1726514994419528
Delta = 0.7457746160667481, sigma = 0.211, Price: 227.52, S-C=216.92000000000002 
call price from BS = 10.51388, Call price from data = 10.6
Error/Difference OP + RE: 0.24864085366199265


0.15546837320594062

In [258]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

def hedging_loopped(interval, strike, df, delta, C_0, S_0, maturity_date, T, r, RE_0, OP_0):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    t = 0
    n = 0

    K = float(strike.split('C')[1])

    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
    
        if row.name == 0:
            continue

        OP = c_1 - c_0

        #print(f'{delta} * ({s_1}-{s_0})')
        RE = -delta * (s_1-s_0)
        # print(f'OP: {OP}, RE: {RE}')
        A = OP + RE
        #print(f'Error/Difference OP + RE: {A}')
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          #print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)
          

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(n-1)
    
    return mse



In [259]:
# Alkuarvot
# Data:
data = pd.read_feather('apple.feather')

# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

start_date = dt.datetime.strptime("2024-09-10", '%Y-%m-%d')
maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')
strikes = [215, 220, 225, 230, 235, 240, 245, 250]
results = []
for k in strikes:
    print(f"---------------- {k}")
    #ATM 2024-09-10
    T = 45/365
    t = 0
    K = k

    strike = f'C{K}'
    C_0 = float(df[strike].iloc[0])
    S_0 = float(df['Underlying'].iloc[0])

    # Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
    r = 0.0497
    eps = np.finfo(float).eps
    delta = calc_delta(S=S_0, K=K, T=T, t=t, r=r, C_0=C_0, eps=np.finfo(float).eps, tol=0.1)
    # print(f"Delta at S_O= {delta}, S_0 = {S_0}, C_0 = {C_0}")

    # Hedging interval
    interval = 1
    mse_s = []
    # init portfolia values
    OP_0 = C_0

    # delta * S_0
    RE_0 = delta * S_0
    
    error = hedging_loopped(interval, strike, df, delta, C_0, S_0, maturity_date, T, r, RE_0, OP_0)
    results.append({"Strike": K, "MSE": error})
    

results_df = pd.DataFrame(results)

---------------- 215
Delta = 0.640503179942022, sigma = 0.27, Price: 220.11, S-C=208.21 
call price from BS = 11.80853, Call price from data = 11.9
Delta = 0.6895761963692004, sigma = 0.263, Price: 222.66, S-C=209.35999999999999 
call price from BS = 13.20473, Call price from data = 13.3
Delta = 0.6800369767221519, sigma = 0.288, Price: 222.77, S-C=208.82000000000002 
call price from BS = 13.85768, Call price from data = 13.95
Delta = 0.679886835042331, sigma = 0.281, Price: 222.5, S-C=209.05 
call price from BS = 13.37422, Call price from data = 13.45
Delta = 0.5735357163228186, sigma = 0.239, Price: 216.32, S-C=208.23999999999998 
call price from BS = 8.00294, Call price from data = 8.08
Delta = 0.5844028779609199, sigma = 0.239, Price: 216.79, S-C=208.54 
call price from BS = 8.1737, Call price from data = 8.25
Delta = 0.6721939746055376, sigma = 0.24, Price: 220.69, S-C=210.04 
call price from BS = 10.55079, Call price from data = 10.65
Delta = 0.8171474356649555, sigma = 0.248, Pr